Dataset notebook: 
- extend missense variant tables with charge "DCharge" and hydrophobicity changes "DChargeHP"
- binding and active site locations from Uniprot

Foldingenergies notebook:
- variant median ddgs across structures

Statscores:
- thresholds and local minima (to be applied to all variants)

Prody:
- conservative Hinge sites

Calculateproperties:
- Loop regions from pdb
- Meta predictor variant prediction and peak (variant)
- Median ddgs (variant)
- Charge, polarity and hydrophobicity change (builds up a solubility picture with sasa) (variant)
- Solvent accessible surface area (residue)
- MoreRONN disorder score (residue)
- Conservation scores (residue)


TO DO:
  - Residue distance to hinge sites, binding sites, active sites
  - ?
  - DNA binding (direct and indirect)
  - Any of the 'others features'
  - EBI catalytic residue roles

LATER ON TO DO:
- Darwinian shift
- Fold structure 3D

FOR LOGISTIC REGRESSION:
- Look at the middle peak in meta predictor scores


In [316]:
import json
import pandas as pd
import numpy as np
import os
import requests
import re
from Bio.PDB import PDBParser
import MDAnalysis as mda
from MDAnalysis.analysis import distances
import freesasa
from Bio.PDB import PDBList

In [440]:
gene = "PTPN11"

# Dictionary of **key** PDB IDs for each gene
pdb_ids = {
    "PTPN11": "2SHP", 
}

# Dictionary of Uniprot IDs for each gene
uniprot_ids = {
    "PTPN11": "Q06124", 
}


chain = "A"
charge_model = "Charge" # or ChargeHP is histidine is positive

In [398]:
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

In [278]:
# Mapping Uniprot/Ensembl <--> PDB residue positions
def get_pdb_uniprot_map(pdb_id, chain="A"):
    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/{pdb_id.lower()}"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()[pdb_id.lower()]["UniProt"]
    
    mapping = []
    for uniprot_id, info in data.items():
        for m in info['mappings']:
            if m['chain_id'] == chain:
                mapping.append({
                    "uniprot_start": m["unp_start"],
                    "uniprot_end": m["unp_end"],
                    "pdb_start": m["start"]["residue_number"],
                    "pdb_end": m["end"]["residue_number"]
                })
    return pd.DataFrame(mapping)


def map_protein_to_pdb(df, map_df,pdb_col="pdb_residue", protein_col="protein_pos"):
    
    df = df.copy()
    df[pdb_col] = None # initialise new column

    for idx, row in df.iterrows():
        pos = row[protein_col]
        hit = map_df[(map_df.uniprot_start <= pos) & (map_df.uniprot_end >= pos)]

        if len(hit) > 0:
            offset = pos - hit.uniprot_start.values[0]
            df.at[idx, pdb_col] = int(hit.pdb_start.values[0] + offset)
        else:
            df.at[idx, pdb_col] = None
    
    return df


def map_pdb_to_protein(df, map_df, pdb_col="pdb_residue", protein_col="AApos"):
    
    df = df.copy()
    df[protein_col] = None  # initialize new column

    for idx, row in df.iterrows():
        pdb_res = row[pdb_col]
        # Find the mapping row where this PDB residue falls
        hit = map_df[(map_df.pdb_start <= pdb_res) & 
                     (map_df.pdb_start + (map_df.uniprot_end - map_df.uniprot_start) >= pdb_res)]

        if len(hit) > 0:
            offset = pdb_res - hit.pdb_start.values[0]
            df.at[idx, protein_col] = int(hit.uniprot_start.values[0] + offset)
        else:
            df.at[idx, protein_col] = None
    
    return df


In [148]:
map_df = get_pdb_uniprot_map(pdb_ids[gene], chain=chain)
print(map_df)

   uniprot_start  uniprot_end  pdb_start  pdb_end
0              1          525          1      525


In [200]:
def get_uniprot_fasta(gene):
    uniprot_id = uniprot_ids[gene]
    url = f'https://www.uniprot.org/uniprot/{uniprot_id}.fasta'
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.text.strip()  # Return full FASTA format
    else:
        return None  


In [193]:
# Download PTPN11 structure from PDB
pdbl = PDBList()
pdb_file = pdbl.retrieve_pdb_file(pdb_ids[gene], pdir='.', file_format='pdb')
print(pdb_file)

"""
# Define new filename
new_file = "./pdb2SHPstructure.pdb"

# Remove old file if exists
if os.path.exists(new_file):
    os.remove(new_file)

# Rename downloaded file
os.rename(pdb_file, new_file)

# Update variable to point to renamed file
pdb_file = new_file
print(pdb_file)


os.rename("./pdb2shp.ent", "./pdb2SHPstructure.pdb")
pdb_file = "./pdb2SHPstructure.pdb"
print(pdb_file)
"""

.\pdb2shp.ent


'\n# Define new filename\nnew_file = "./pdb2SHPstructure.pdb"\n\n# Remove old file if exists\nif os.path.exists(new_file):\n    os.remove(new_file)\n\n# Rename downloaded file\nos.rename(pdb_file, new_file)\n\n# Update variable to point to renamed file\npdb_file = new_file\nprint(pdb_file)\n\n\nos.rename("./pdb2shp.ent", "./pdb2SHPstructure.pdb")\npdb_file = "./pdb2SHPstructure.pdb"\nprint(pdb_file)\n'

In [184]:
"""
def get_PDBfile(gene, pdb_dir="pdb_files", parse=True):
    
    pdb_id = pdb_ids[gene]#.lower()
    os.makedirs(pdb_dir, exist_ok=True)
    pdb_file = os.path.join(pdb_dir, f"{pdb_id}.pdb")
    
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    r = requests.get(url)
    if r.status_code != 200:
        raise ValueError(f"Failed to download PDB {pdb_id}")
    
    with open(pdb_file, "w") as f:
        f.write(r.text)
    
    if parse:
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure(pdb_id, pdb_file)
        return structure
    else:
        return pdb_file
"""

#### Define parts of the gene we are interested in - domains, sites, loops, hinges

In [279]:
def get_loops_for_gene(gene_name, chain_id="A", pdb_file=None):

    ss_regions = []
    residues = []

    with open(pdb_file, "r") as f:
        for line in f:
            if line.startswith("HELIX") and line[19] == chain_id:
                start = int(line[21:25])
                end = int(line[33:37])
                ss_regions.append((start, end))
            elif line.startswith("SHEET") and line[21] == chain_id:
                start = int(line[22:26])
                end = int(line[33:37])
                ss_regions.append((start, end))
            elif line.startswith("ATOM") and line[21] == chain_id:
                resnum = int(line[22:26])
                if resnum not in residues:
                    residues.append(resnum)

    residues.sort()
    loops = []
    loop_start = None
    for res in residues:
        in_ss = any(start <= res <= end for start, end in ss_regions)
        if not in_ss:
            if loop_start is None:
                loop_start = res
        else:
            if loop_start is not None:
                loops.append((loop_start, res - 1))
                loop_start = None
    if loop_start is not None:
        loops.append((loop_start, residues[-1]))

    return loops


In [280]:
# Load Uniprot features
with open(f"{gene}_uniprotfeatures.json", "r") as f:
    uniprot_features = json.load(f)

domains = uniprot_features["domains"]
sites = uniprot_features["sites"]
others = uniprot_features["other features"]

# what do I want from 'others'??


In [476]:
domains

[['SH2 1', 6, 102, '#66c2a5'],
 ['SH2 2', 112, 216, '#66c2a5'],
 ['Tyrosine-protein phosphatase', 247, 517, '#66c2a5'],
 ['Disordered', 548, 571, '#fc8d62']]

In [308]:
# Load hinges
prodyhinges = pd.read_csv(f"{gene}_prodyhinges.csv")

In [346]:
def uniprotfeatures_pdbloops_prodyhinges_todf(
    uniprot_features,
    map_df,
    gene,
    chain_id=chain,
    pdb_file=pdb_file,
    hinge_file_template="{gene}_prodyhinges.csv"
):

    import pandas as pd

    rows = []

    # ---- UniProt: Domains & Regions ----
    for name, start, end, color in uniprot_features.get("domains", []):
        rows.append({
            "name": name,
            "type": "Domain" if color == "#66c2a5" else "Region",
            "protein_start": start,
            "protein_end": end,
            "color": color
        })

    # ---- UniProt: Sites ----
    for name, start, end, color in uniprot_features.get("sites", []):
        rows.append({
            "name": name or "",
            "type": "Active site" if color == "#e78ac3" else "Binding site",
            "protein_start": start,
            "protein_end": end,
            "color": color
        })

    df = pd.DataFrame(rows)

    # ---- Map UniProt positions to PDB ----
    if len(df) > 0:
        df["pdb_start"] = map_protein_to_pdb(
            df[["protein_start"]].rename(columns={"protein_start": "protein_pos"}),
            map_df,
            pdb_col="pdb_residue",
            protein_col="protein_pos"
        )["pdb_residue"]

        df["pdb_end"] = map_protein_to_pdb(
            df[["protein_end"]].rename(columns={"protein_end": "protein_pos"}),
            map_df,
            pdb_col="pdb_residue",
            protein_col="protein_pos"
        )["pdb_residue"]
    else:
        df["pdb_start"] = None
        df["pdb_end"] = None

    # ---- Add loops (PDB features, mapped to protein coordinates) ----
    loop_regions = get_loops_for_gene(gene, chain_id, pdb_file=pdb_file)

    loop_rows = []
    for loop_start, loop_end in loop_regions:
        # Map PDB -> protein
        tmp_start = pd.DataFrame({"pdb_residue": [loop_start]})
        tmp_end   = pd.DataFrame({"pdb_residue": [loop_end]})

        protein_start = map_pdb_to_protein(
            tmp_start, map_df, pdb_col="pdb_residue", protein_col="protein_pos"
        )["protein_pos"].iloc[0]

        protein_end = map_pdb_to_protein(
            tmp_end, map_df, pdb_col="pdb_residue", protein_col="protein_pos"
        )["protein_pos"].iloc[0]

        loop_rows.append({
            "name": "",
            "type": "Loop",
            "protein_start": protein_start,
            "protein_end": protein_end,
            "pdb_start": loop_start,
            "pdb_end": loop_end,
            "color": "#d9d9d9"
        })

    loops_df = pd.DataFrame(loop_rows)

    # ---- Add hinges from ProDy ----
    hinge_file = hinge_file_template.format(gene=gene)
    hinges = pd.read_csv(hinge_file)

    hinges["name"] = ""
    hinges["type"] = "Hinge"
    hinges["pdb_start"] = hinges["pdb_residue"] - 4 # as defined the change to be 4 residues consistent
    hinges["pdb_end"] = hinges["pdb_residue"] + 4

    # Map PDB -> protein
    tmp_start = hinges[["pdb_start"]].rename(columns={"pdb_start": "pdb_residue"})
    tmp_end   = hinges[["pdb_end"]].rename(columns={"pdb_end": "pdb_residue"})

    hinges["protein_start"] = map_pdb_to_protein(tmp_start, map_df, pdb_col="pdb_residue", protein_col="protein_pos")["protein_pos"]
    hinges["protein_end"]   = map_pdb_to_protein(tmp_end, map_df, pdb_col="pdb_residue", protein_col="protein_pos")["protein_pos"]

    hinges["color"] = "#000000"

    hinges = hinges[[
        "name", "type", "protein_start", "protein_end",
        "pdb_start", "pdb_end", "color"
    ]]

    # ---- Combine all features ----
    combined = pd.concat([df, loops_df, hinges], ignore_index=True)

    return combined[
        ["name", "type", "protein_start", "protein_end", "pdb_start", "pdb_end", "color"]
    ]


In [353]:
features_uniprotpdbprody_df = uniprotfeatures_pdbloops_prodyhinges_todf(
    uniprot_features,
    map_df,
    gene,
    chain_id="A",
    pdb_file=pdb_file,
    hinge_file_template="{gene}_prodyhinges.csv"
)



In [354]:
features_uniprotpdbprody_df

,name,type,protein_start,protein_end,pdb_start,pdb_end,color
0,SH2 1,Domain,6,102,6,102,#66c2a5
1,SH2 2,Domain,112,216,112,216,#66c2a5
2,Tyrosine-protein phosphatase,Domain,247,517,247,517,#66c2a5
3,Disordered,Region,548,571,None,None,#fc8d62
4,Phosphocysteine intermediate,Active site,459,459,459,459,#e78ac3
5,,Binding site,425,425,425,425,#a6d854
6,,Binding site,459,465,459,465,#a6d854
7,,Binding site,506,506,506,506,#a6d854
8,,Loop,2,12,2,12,#d9d9d9
9,,Loop,24,28,24,28,#d9d9d9


In [312]:
features_uniprotpdbprody_df.to_csv(f"{gene}_features_uniprotpdbprody.csv", index=False)

#### Calculate residue min dist

In [360]:
u = mda.Universe(pdb_file)
DNA = u.select_atoms("nucleic")
proteins = u.select_atoms("protein and chainID {chain}") # should really do calcs below for every chain and pick minimum?




In [370]:
# Distance helper
def min_distance_to_feature(protein_ag, feature_ag):
    dist_matrix = distances.distance_array(
        protein_ag.positions,
        feature_ag.positions
    )
    min_per_atom = np.min(dist_matrix, axis=1)

    min_residue_dist = {}
    for resid, d in zip(protein_ag.resids, min_per_atom):
        if resid not in min_residue_dist:
            min_residue_dist[resid] = d
        else:
            min_residue_dist[resid] = min(min_residue_dist[resid], d)

    return min_residue_dist

In [477]:
"""
# Storage
feature_atomgroups = {}
feature_distance_maps = {}

# Counters for autogenerated names
counters = {
    "Active site": 1,
    "Binding site": 1,
    "Loop": 1,
    "Hinge": 1
}

# Proteins atom group
proteins = u.select_atoms(f"protein and chainID {chain_id}")

for idx, row in features_uniprotpdbprody_df.iterrows():

    if row['pdb_start'] is None or row['pdb_end'] is None:
        continue

    pdb_start = row['pdb_start']
    pdb_end   = row['pdb_end']

    # Naming logic
    if row['type'] in ['Domain', 'Region']:
        name_clean = row['name'].replace(' ', '_').replace('-', '_')
    elif row['type'] in ['Active site', 'Binding site', 'Loop', 'Hinge']:
        base = row['type'].replace(' ', '').lower()
        name_clean = f"{base}{counters[row['type']]}"
        counters[row['type']] += 1
    else:
        continue  # skip irrelevant rows

    # Atom selection
    selection_str = f"protein and chainID {chain_id} and resnum {pdb_start}:{pdb_end}"
    feature_ag = u.select_atoms(selection_str)

    # Store AtomGroup
    feature_atomgroups[name_clean] = feature_ag

    # --- Distance calculation for non-domains ---
    if row['type'] != 'Domain':
        feature_distance_maps[name_clean + "DIST"] = min_distance_to_feature(proteins, feature_ag)
    else:
        # --- Domain indicator: binary column in dataframe ---
        calculatedproperties_df[name_clean] = calculatedproperties_df["AApos"].apply(
            lambda x: 1 if pdb_start <= x <= pdb_end else 0
        )

    print(f"Processed {name_clean} ({row['type']}): {selection_str}")
"""

Processed SH2_1 (Domain): protein and chainID A and resnum 6:102
Processed SH2_2 (Domain): protein and chainID A and resnum 112:216
Processed Tyrosine_protein_phosphatase (Domain): protein and chainID A and resnum 247:517
Processed activesite1 (Active site): protein and chainID A and resnum 459:459
Processed bindingsite1 (Binding site): protein and chainID A and resnum 425:425
Processed bindingsite2 (Binding site): protein and chainID A and resnum 459:465
Processed bindingsite3 (Binding site): protein and chainID A and resnum 506:506
Processed loop1 (Loop): protein and chainID A and resnum 2:12
Processed loop2 (Loop): protein and chainID A and resnum 24:28
Processed loop3 (Loop): protein and chainID A and resnum 34:39
Processed loop4 (Loop): protein and chainID A and resnum 48:49
Processed loop5 (Loop): protein and chainID A and resnum 58:73
Processed loop6 (Loop): protein and chainID A and resnum 84:118
Processed loop7 (Loop): protein and chainID A and resnum 130:134
Processed loop8 (

In [378]:
# Storage
feature_atomgroups = {}
feature_distance_maps = {}

# Counters for autogenerated names
counters = {
    "Active site": 1,
    "Binding site": 1,
    "Loop": 1,
    "Hinge": 1
}

# Single unified loop
for idx, row in features_uniprotpdbprody_df.iterrows():

    if row['pdb_start'] is None or row['pdb_end'] is None:
        continue

    pdb_start = row['pdb_start']
    pdb_end   = row['pdb_end']

    # Naming logic
    if row['type'] == 'Domain':
        # store AtomGroup but DO NOT calculate distances
        name_clean = row['name'].replace(' ', '_').replace('-', '_')
        feature_atomgroups[name_clean] = u.select_atoms(selection_str)
        print(f"Stored domain {name_clean} (no distances calculated)")
        continue     # skip distance calculation

    elif row['type'] in ['Active site', 'Binding site', 'Loop', 'Hinge']:
        base = row['type'].replace(' ', '').lower()
        name_clean = f"{base}{counters[row['type']]}"
        counters[row['type']] += 1

    else:
        continue  # ignore anything else

    # Atom selection
    selection_str = f"protein and chainID {chain_id} and resnum {pdb_start}:{pdb_end}"
    feature_ag = u.select_atoms(selection_str)

    # Store AtomGroup + distance map
    feature_atomgroups[name_clean] = feature_ag
    feature_distance_maps[name_clean+ "Dist"] = min_distance_to_feature(proteins, feature_ag)

    print(f"Created {name_clean} ({row['type']}): {selection_str}")

Stored domain SH2_1 (no distances calculated)
Stored domain SH2_2 (no distances calculated)
Stored domain Tyrosine_protein_phosphatase (no distances calculated)
Created activesite1 (Active site): protein and chainID A and resnum 459:459
Created bindingsite1 (Binding site): protein and chainID A and resnum 425:425
Created bindingsite2 (Binding site): protein and chainID A and resnum 459:465
Created bindingsite3 (Binding site): protein and chainID A and resnum 506:506
Created loop1 (Loop): protein and chainID A and resnum 2:12
Created loop2 (Loop): protein and chainID A and resnum 24:28
Created loop3 (Loop): protein and chainID A and resnum 34:39
Created loop4 (Loop): protein and chainID A and resnum 48:49
Created loop5 (Loop): protein and chainID A and resnum 58:73
Created loop6 (Loop): protein and chainID A and resnum 84:118
Created loop7 (Loop): protein and chainID A and resnum 130:134
Created loop8 (Loop): protein and chainID A and resnum 140:146
Created loop9 (Loop): protein and cha

In [379]:
feature_distance_maps

{'activesite1Dist': {2: 23.845230012715646,
  3: 25.7958070084979,
  4: 18.54208622739504,
  5: 27.09355746957735,
  6: 22.1114368122502,
  7: 22.464944883698504,
  8: 26.51616351812553,
  9: 26.523680707498418,
  10: 28.577163710965205,
  11: 29.250416330210804,
  12: 31.824558109101947,
  13: 31.513056299278297,
  14: 34.1286255165995,
  15: 34.27665910099209,
  16: 31.693728583135577,
  17: 34.26123333230297,
  18: 36.95791345953947,
  19: 34.71857294162375,
  20: 32.57841030776983,
  21: 37.804409009864145,
  22: 39.87836922343331,
  23: 38.771196964843256,
  24: 38.03723358555137,
  25: 36.672544235260474,
  26: 36.39333679032996,
  27: 32.57549281294585,
  28: 32.16294698158664,
  29: 26.770067822486315,
  30: 26.861259959213918,
  31: 23.69030673700553,
  32: 23.39779664178989,
  33: 23.08066478530814,
  34: 22.613564924605253,
  35: 25.249675472550692,
  36: 24.115272702511216,
  37: 20.2701981730301,
  38: 18.899053316438923,
  39: 16.783605338872754,
  40: 18.148999070185823,

In [389]:
feature_atomgroups

{'SH2_1': <AtomGroup with 88 atoms>,
 'SH2_2': <AtomGroup with 88 atoms>,
 'Tyrosine_protein_phosphatase': <AtomGroup with 88 atoms>,
 'activesite1': <AtomGroup with 7 atoms>,
 'bindingsite1': <AtomGroup with 9 atoms>,
 'bindingsite2': <AtomGroup with 57 atoms>,
 'bindingsite3': <AtomGroup with 12 atoms>,
 'loop1': <AtomGroup with 131 atoms>,
 'loop2': <AtomGroup with 35 atoms>,
 'loop3': <AtomGroup with 52 atoms>,
 'loop4': <AtomGroup with 16 atoms>,
 'loop5': <AtomGroup with 154 atoms>,
 'loop6': <AtomGroup with 352 atoms>,
 'loop7': <AtomGroup with 44 atoms>,
 'loop8': <AtomGroup with 62 atoms>,
 'loop9': <AtomGroup with 59 atoms>,
 'loop10': <AtomGroup with 19 atoms>,
 'loop11': <AtomGroup with 79 atoms>,
 'loop12': <AtomGroup with 222 atoms>,
 'loop13': <AtomGroup with 10 atoms>,
 'loop14': <AtomGroup with 48 atoms>,
 'loop15': <AtomGroup with 17 atoms>,
 'loop16': <AtomGroup with 245 atoms>,
 'loop17': <AtomGroup with 32 atoms>,
 'loop18': <AtomGroup with 33 atoms>,
 'loop19': <A

#### MoreRONN disorder score - SHOULD DOWNLOAD LOCALLY FOR LINUX

In [201]:
# Example usage
fastasequence = get_uniprot_fasta(gene)

print(fastasequence)

>sp|Q06124|PTN11_HUMAN Tyrosine-protein phosphatase non-receptor type 11 OS=Homo sapiens OX=9606 GN=PTPN11 PE=1 SV=3
MTSRRWFHPNITGVEAENLLLTRGVDGSFLARPSKSNPGDFTLSVRRNGAVTHIKIQNTG
DYYDLYGGEKFATLAELVQYYMEHHGQLKEKNGDVIELKYPLNCADPTSERWFHGHLSGK
EAEKLLTEKGKHGSFLVRESQSHPGDFVLSVRTGDDKGESNDGKSKVTHVMIRCQELKYD
VGGGERFDSLTDLVEHYKKNPMVETLGTVLQLKQPLNTTRINAAEIESRVRELSKLAETT
DKVKQGFWEEFETLQQQECKLLYSRKEGQRQENKNKNRYKNILPFDHTRVVLHDGDPNEP
VSDYINANIIMPEFETKCNNSKPKKSYIATQGCLQNTVNDFWRMVFQENSRVIVMTTKEV
ERGKSKCVKYWPDEYALKEYGVMRVRNVKESAAHDYTLRELKLSKVGQGNTERTVWQYHF
RTWPDHGVPSDPGGVLDFLEEVHHKQESIMDAGPVVVHCSAGIGRTGTFIVIDILIDIIR
EKGVDCDIDVPKTIQMVRSQRSGMVQTEAQYRFIYMAVQHYIETLQRRIEEEQKSKRKGH
EYTNIKYSLADQTSGDQSPLPPCTPTPPCAEMREDSARVYENVGLMQQQKSFR


In [209]:
def parse_moreronn_scores(filepath):
    """
    Parse MoréRonn result file to extract:
        - Amino acid letters
        - MoréRonn disorder scores
        - AA position (1-indexed)
    Returns a DataFrame.
    """

    aa_list = []
    score_list = []

    # Catch lines like "A 0.12345"
    pattern = re.compile(r"^([A-Z])\s+([0-9]*\.[0-9]+)$")

    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()

            m = pattern.match(line)
            if m:
                aa = m.group(1)
                score = float(m.group(2))
                aa_list.append(aa)
                score_list.append(score)

    # Build DataFrame
    df = pd.DataFrame({
        "AAfrom": aa_list,
        "MoreRonn": score_list
    })

    # Add AA position index (1-based)
    df["AApos"] = df.index + 1

    return df


In [210]:
moreronn_df = parse_moreronn_scores(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\sp_Q06124_PTN11_HUMAN Tyrosine-protein phosphatase non-receptor type 11 OS=Homo sapiens OX=9606 GN=PTPN11 PE=1 SV=3.txt")
moreronn_df.head()


,AAfrom,MoreRonn,AApos
0,M,0.366789,1
1,T,0.350520,2
2,S,0.343691,3
3,R,0.346683,4
4,R,0.333852,5


In [211]:
moreronn_df.to_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_moreronn.csv", index=False)


#### Conservation score (use DS BigWig as an automated version?)
Using ConSur (on Google colab as https://consurf.tau.ac.il/ down due to technical issues) with default parametres except mode = PDB
https://colab.research.google.com/drive/1PhDXX7k12oUsV6T_xkXC3Rm9R99e7tHz#scrollTo=zBTwgYcIK6Ul 

In [262]:
def parse_consurf_per_residue(file_path): 

    aa_list = []
    pos_list = []
    score_list = []
    grade_list = []

    with open(file_path, 'r') as f:
        header_found = False
        for line in f:
            line = line.strip()
            if not line:
                continue

            # Detect header line
            if line.startswith("POS") and "SEQ" in line:
                header_found = True
                continue

            if not header_found:
                continue

            # Split line on whitespace, maxsplit=9 to capture RESIDUE VARIETY as last column
            cols = re.split(r'\s+', line, maxsplit=9)
            if len(cols) < 5:
                continue  # skip malformed lines

            # Only parse lines where first column is a digit (residue number)
            if not cols[0].isdigit():
                continue

            pos = int(cols[0])
            aa = cols[1]
            score = float(cols[2])
            grade = int(cols[3])

            pos_list.append(pos)
            aa_list.append(aa)
            score_list.append(score)
            grade_list.append(grade)

    df = pd.DataFrame({
        'AApos': pos_list, # PDBpos is use structure
        'AAfrom': aa_list,
        'ConSurfScore': score_list,
        'ConSurfGrade': grade_list,
    })

    return df


In [264]:
#uses Uniprot sequence, what difference does adding structure make - issue if there is a substitution in the structure?
# 9=conserved, 1=variable
consurf_df = parse_consurf_per_residue(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\Consurf_Outputs_Q06124\Q06124_consurf_grades.txt")
print(consurf_df.head())


   AApos AAfrom  ConSurfScore  ConSurfGrade
0      1      M        -0.591             8
1      2      T         0.792             3
2      3      S        -0.512             7
3      4      R        -0.259             6
4      5      R        -0.785             9


In [248]:
# Using Uniprot sequence above
#consurf_df = map_pdb_to_protein(consurf_df, map_df, pdb_col="PDBpos", protein_col="AApos")


In [265]:
consurf_df.to_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_ConSurf.csv", index=False)


#### Solvent accessible surface area

In [191]:
with open(pdb_file) as f:
    lines = f.readlines()
print(lines[:10])

['HEADER    TYROSINE PHOSPHATASE                    01-DEC-97   2SHP              \n', 'TITLE     TYROSINE PHOSPHATASE SHP-2                                            \n', 'COMPND    MOL_ID: 1;                                                            \n', 'COMPND   2 MOLECULE: SHP-2;                                                     \n', 'COMPND   3 CHAIN: A, B;                                                         \n', 'COMPND   4 SYNONYM: SYP, SHPTP-2;                                               \n', 'COMPND   5 EC: 3.1.3.48;                                                        \n', 'COMPND   6 ENGINEERED: YES;                                                     \n', 'COMPND   7 MUTATION: YES                                                        \n', 'SOURCE    MOL_ID: 1;                                                            \n']


In [79]:
with open(pdb_file) as f:  
    lines = f.readlines()

# Check if any ATOM lines exist
atom_lines = [line for line in lines if line.startswith("ATOM")]
print(f"Number of ATOM lines: {len(atom_lines)}")
print(atom_lines[:5])  # show first 5 atoms


Number of ATOM lines: 9804
['ATOM      1  N   LYS A   2      17.460 -50.947  44.421  1.00 48.96           N  \n', 'ATOM      2  CA  LYS A   2      16.169 -50.459  43.825  1.00 49.25           C  \n', 'ATOM      3  C   LYS A   2      16.308 -50.237  42.299  1.00 49.37           C  \n', 'ATOM      4  O   LYS A   2      16.608 -49.118  41.852  1.00 48.89           O  \n', 'ATOM      5  CB  LYS A   2      15.728 -49.142  44.503  1.00 48.37           C  \n']


In [81]:
# Load the PDB structure
structure = freesasa.Structure(pdb_file)

# Use default parameters (Lee-Richards algorithm)
result = freesasa.calc(structure)

In [101]:
sasa_data = []

# Iterate over chains and residues
for chain_id, residues in result.residueAreas().items():
    for res_num, area in residues.items():
        sasa_data.append({
            "Chain": chain_id,
            "ResNum": int(res_num),  # convert to integer
            "Total_SASA": area.total,
            "Polar_SASA": area.polar,
            "Apolar_SASA": area.apolar,
            "MainChain_SASA": area.mainChain,
            "SideChain_SASA": area.sideChain
        })

# Convert to DataFrame for easy viewing
sasa_df = pd.DataFrame(sasa_data)

# Select chain
sasa_df = sasa_df.copy()
sasa_df = sasa_df[sasa_df["Chain"] == chain].reset_index(drop=True)

# Sort by residue number
sasa_df = sasa_df.sort_values(["ResNum"]).reset_index(drop=True)

sasa_df.head()

,Chain,ResNum,Total_SASA,Polar_SASA,Apolar_SASA,MainChain_SASA,SideChain_SASA
0,A,2,70.353038,49.220175,21.132863,34.116774,36.236264
1,A,3,68.173297,25.410800,42.762496,13.993382,54.179914
2,A,4,4.758575,4.758575,0.000000,0.000000,4.758575
3,A,5,40.571488,20.133294,20.438194,0.997283,39.574205
4,A,6,3.507165,0.003346,3.503820,0.000000,3.507165


In [102]:
sasa_df = map_pdb_to_protein(sasa_df, map_df, pdb_col="ResNum", protein_col="AApos")

In [104]:
sasa_df.head()

,Chain,ResNum,Total_SASA,Polar_SASA,Apolar_SASA,MainChain_SASA,SideChain_SASA,AApos
0,A,2,70.353038,49.220175,21.132863,34.116774,36.236264,2
1,A,3,68.173297,25.410800,42.762496,13.993382,54.179914,3
2,A,4,4.758575,4.758575,0.000000,0.000000,4.758575,4
3,A,5,40.571488,20.133294,20.438194,0.997283,39.574205,5
4,A,6,3.507165,0.003346,3.503820,0.000000,3.507165,6


In [103]:
sasa_df.to_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_sasa.csv", index=False)


#### Metapredictors for annotations

In [9]:
with open(f"{gene}_thresholds.json", "r") as f:
    thresholds = json.load(f)

with open(f"{gene}_localmins.json", "r") as f:
    localmins = json.load(f)


In [52]:
def annotate_scores(
    df,
    thresholds_dict,
    localmins_dict,
    Threshold=True,
    Peaks=True
):

    df = df.copy()

    # Score renaming map
    label_map = {
        "BayesDel_noAF": "BD",
        "BayesDel_addAF": "BDAF"
    }

    # Threshold prediction
    if Threshold:
        for score_name, threshold in thresholds_dict.items():

            short = label_map.get(score_name, score_name)   # fallback = same name
            pred_col = f"pred{short}"

            df[pred_col] = np.where(
                df[score_name] >= threshold,
                "Pathogenic",
                np.where(df[score_name].notna(), "Benign", None)
            )

            df[f"{pred_col}num"] = df[pred_col].map({
                "Benign": 0,
                "Pathogenic": 1
            })

    # Peak-based classification
    if Peaks:
        for score_name, mins in localmins_dict.items():

            short = label_map.get(score_name, score_name)
            new_col = f"{short}peak"

            m1, m2 = sorted(mins)

            df[new_col] = np.select(
                [
                    df[score_name] < m1,
                    (df[score_name] >= m1) & (df[score_name] <= m2),
                    df[score_name] > m2
                ],
                [0, 0.5, 1],
                default=np.nan
            )

    return df


In [53]:
revel_bayesdel_agg = pd.read_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_REVEL_BayesDel_agg.csv")


In [54]:
revel_bayesdel_agg_labeled = annotate_scores(
    revel_bayesdel_agg,
    thresholds_dict=thresholds,
    localmins_dict=localmins,
    Threshold=True,
    Peaks=True
)

In [55]:
revel_bayesdel_agg_labeled.sort_values(by="AApos").head()


,AAfrom,AAto,AApos,REVEL,BayesDel_noAF,BayesDel_addAF,predREVEL,predREVELnum,predBD,predBDnum,predBDAF,predBDAFnum,REVELpeak,BDpeak,BDAFpeak
1923,M,T,1.0,0.315,0.660000,0.660000,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0
1909,M,R,1.0,0.202,0.660000,0.660000,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0
1882,M,K,1.0,0.294,0.660000,0.660000,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0
1868,M,I,1.0,0.222,0.660000,0.660000,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0
3021,T,R,2.0,0.063,-0.222939,-0.222939,Benign,0,Benign,0,Benign,0,0.0,0.0,0.0


In [56]:
revel_bayesdel_agg_labeled.to_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_REVEL_BayesDel_agg_labeled.csv", index=False)


# Start creating df of calculated properties for all possible missense variants + calculate charge

In [478]:
foldingenergy_df = pd.read_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_foldingenergy_df.csv")
revel_bayesdel_agg_labeled = pd.read_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_REVEL_BayesDel_agg_labeled.csv")
sasa_df = pd.read_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_sasa.csv")
moreronn_df = pd.read_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_moreronn.csv")
consurf_df = pd.read_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_ConSurf.csv")



In [479]:
calculatedproperties_df = (revel_bayesdel_agg_labeled.merge( # start with meta predictors as across the entire sequence
    foldingenergy_df, 
    on=['AApos', 'AAfrom', 'AAto'],
    how='left')
    .sort_values(["AApos", "AAto"])
    .reset_index(drop=True)
    #.drop(columns=["predREVEL", "predBD", "predBDAF"])                      
    )

In [480]:
calculatedproperties_df = (calculatedproperties_df.copy()
    .merge(sasa_df,on=['AApos'],how='left')
    .drop(columns=["Chain", "ResNum"])
    )


In [481]:
calculatedproperties_df = (calculatedproperties_df.copy()
    .merge(moreronn_df,on=['AApos','AAfrom'],how='left')
    )


In [482]:
calculatedproperties_df = (calculatedproperties_df.copy()
    .merge(consurf_df,on=['AApos','AAfrom'],how='left') # 2SHP structure in ConSurf has a difference AA in AApos 2 so haven't inc ,'AAfrom'
    #.drop(columns=["PDBpos"])
    )



In [483]:
for feat_name, dist_map in feature_distance_maps.items():
    calculatedproperties_df[feat_name] = calculatedproperties_df["AApos"].map(dist_map)


In [484]:
def abbreviate_domain(name):
    # If name is already uppercase and/or contains numbers, keep as is
    name_clean = name.replace(" ", "_")
    if re.fullmatch(r'[A-Z0-9]+', name_clean.replace(" ", "").replace("-", "").replace("_", "")):
        return name_clean.upper()
    
    # Otherwise, abbreviate: take first letter of each word
    parts = re.split(r'[\s_-]+', name_clean)
    abbrev = ''.join([p[0].upper() for p in parts if p])
    return abbrev


for idx, row in features_uniprotpdbprody_df.iterrows():

    if row['pdb_start'] is None or row['pdb_end'] is None:
        continue

    if row['type'] == 'Domain':
        domain_abbrev = abbreviate_domain(row['name'])
        pdb_start = row['pdb_start']
        pdb_end = row['pdb_end']

        # Binary column: 1 if residue is in domain, 0 otherwise
        calculatedproperties_df["Domain_" + domain_abbrev] = calculatedproperties_df["AApos"].apply(
            lambda x: 1 if pdb_start <= x <= pdb_end else 0
        )



#### Finally, compute change in charge, polarity and hydrophobicity
###### ? alsovolume/size, aromatic v non-aromatic change

In [485]:
aacharge = {
         'C': 0, 'D': -1, 'S': 0, 'Q': 0, 'K': 1,
         'I': 0, 'P': 0, 'T': 0, 'F': 0, 'N': 0, 
         'G': 0, 'H': 0, 'L': 0, 'R': 1, 'W': 0, 
         'A': 0, 'V':0, 'E': -1, 'Y': 0, 'M': 0 }

aachargeHP = {
         'C': 0, 'D': -1, 'S': 0, 'Q': 0, 'K': 1,
         'I': 0, 'P': 0, 'T': 0, 'F': 0, 'N': 0, 
         'G': 0, 'H': 1, 'L': 0, 'R': 1, 'W': 0, 
         'A': 0, 'V':0, 'E': -1, 'Y': 0, 'M': 0 }

polarity = {
    'A': 8.1,  'R': 10.5, 'N': 11.6, 'D': 13.0,
    'C': 5.5,  'Q': 10.5, 'E': 12.3, 'G': 9.0,
    'H': 10.4, 'I': 5.2,  'L': 4.9,  'K': 11.3,
    'M': 5.7,  'F': 5.2,  'P': 8.0,  'S': 9.2,
    'T': 8.6,  'W': 5.4,  'Y': 6.2,  'V': 5.9,}  # Grantham 1974

polarity_category = {
    # Nonpolar / hydrophobic
    'A': "N", 'V': "N", 'L': "N", 'I': "N",
    'M': "N", 'F': "N", 'W': "N", 'P': "N",
    'G': "N",

    # Polar uncharged
    'S': "P", 'T': "P", 'C': "P",
    'N': "P", 'Q': "P", 'Y': "P",

    # Positive (basic)
    'K': "pos", 'R': "pos", 'H': "pos",

    # Negative (acidic)
    'D': "neg", 'E': "neg"
}

# Kyte-Doolittle hydrophobicity scale 
# (more positive = more hydrophobic = less soluble, more negative = more hydrophilic = more soluble)
hydrophobicity = { 
    'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5,
    'C': 2.5, 'Q': -3.5, 'E': -3.5, 'G': -0.4,
    'H': -3.2, 'I': 4.5, 'L': 3.8, 'K': -3.9,
    'M': 1.9, 'F': 2.8, 'P': -1.6, 'S': -0.8,
    'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2
}

def compute_DchargeDpolarityDHydrophobicity(df, charge_model=charge_model):
    df = df.copy()  

    # Compute charge change based on the selected model
    if charge_model == "Charge":
        df["DCharge"] = df.apply(
            lambda row: aacharge.get(row["AAto"]) - aacharge.get(row["AAfrom"]),
            axis=1
        )
    elif charge_model == "ChargeHP":
        df["DChargeHP"] = df.apply(
            lambda row: aachargeHP.get(row["AAto"]) - aachargeHP.get(row["AAfrom"]),
            axis=1
        )
    else:
        raise ValueError("charge_model must be 'Charge' or 'ChargeHP'")

    # Polarity change
    df["DPolarity"] = df.apply(
        lambda row: polarity.get(row["AAto"]) - polarity.get(row["AAfrom"]),
        axis=1
    )

    # Polarity category change
    df["DPolarityCat"] = df.apply(
        lambda row: f"{polarity_category.get(row['AAfrom'])}"
                    f"{polarity_category.get(row['AAto'])}",
        axis=1
    )

    # Hydrophobicity change
    df["DHydrophobicity"] = df.apply(
        lambda row: hydrophobicity.get(row["AAto"]) - hydrophobicity.get(row["AAfrom"]),
        axis=1
    )

    return df



In [486]:
calculatedproperties_df = compute_DchargeDpolarityDHydrophobicity(calculatedproperties_df,charge_model=charge_model)

In [487]:
calculatedproperties_df.head(50)

,AAfrom,AAto,AApos,REVEL,BayesDel_noAF,BayesDel_addAF,predREVEL,predREVELnum,predBD,predBDnum,predBDAF,predBDAFnum,REVELpeak,BDpeak,BDAFpeak,medianDDG,Total_SASA,Polar_SASA,Apolar_SASA,MainChain_SASA,SideChain_SASA,MoreRonn,ConSurfScore,ConSurfGrade,activesite1DIST,bindingsite1DIST,bindingsite2DIST,bindingsite3DIST,loop1DIST,loop2DIST,loop3DIST,loop4DIST,loop5DIST,loop6DIST,loop7DIST,loop8DIST,loop9DIST,loop10DIST,loop11DIST,loop12DIST,loop13DIST,loop14DIST,loop15DIST,loop16DIST,loop17DIST,loop18DIST,loop19DIST,loop20DIST,loop21DIST,loop22DIST,loop23DIST,loop24DIST,loop25DIST,loop26DIST,loop27DIST,loop28DIST,loop29DIST,hinge1DIST,hinge2DIST,Domain_SH2_1,Domain_SH2_2,Domain_TPP,DCharge,DPolarity,DPolarityCat,DHydrophobicity
0,M,I,1.0,0.2220,0.660000,0.660000,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0,0.264062,NaN,NaN,NaN,NaN,NaN,0.366789,-0.591,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,-0.5,NN,2.6
1,M,K,1.0,0.2940,0.660000,0.660000,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0,0.188726,NaN,NaN,NaN,NaN,NaN,0.366789,-0.591,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,5.6,Npos,-5.8
2,M,R,1.0,0.2020,0.660000,0.660000,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0,0.375042,NaN,NaN,NaN,NaN,NaN,0.366789,-0.591,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,4.8,Npos,-6.4
3,M,T,1.0,0.3150,0.660000,0.660000,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0,0.996721,NaN,NaN,NaN,NaN,NaN,0.366789,-0.591,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,2.9,NP,-2.6
4,T,A,2.0,0.0540,-0.279529,-0.279529,Benign,0,Benign,0,Benign,0,0.0,0.0,0.0,-0.139115,70.353038,49.220175,21.132863,34.116774,36.236264,0.350520,0.792,3.0,23.845230,28.254788,16.440256,14.313013,0.0,10.719504,17.479466,19.159104,7.792541,6.411087,22.258059,26.465790,29.753970,22.173557,13.683550,16.240129,21.365588,3.755277,18.185293,12.235307,23.640460,22.629094,30.619216,24.601382,39.540737,37.286003,43.923780,25.572965,29.902617,16.440256,20.706048,11.813074,37.976353,6.411087,19.448170,0,0,0,0,-0.5,PN,2.5
5,T,I,2.0,0.2130,0.010807,0.010807,Benign,0,Benign,0,Benign,0,0.0,0.0,0.0,0.148935,70.353038,49.220175,21.132863,34.116774,36.236264,0.350520,0.792,3.0,23.845230,28.254788,16.440256,14.313013,0.0,10.719504,17.479466,19.159104,7.792541,6.411087,22.258059,26.465790,29.753970,22.173557,13.683550,16.240129,21.365588,3.755277,18.185293,12.235307,23.640460,22.629094,30.619216,24.601382,39.540737,37.286003,43.923780,25.572965,29.902617,16.440256,20.706048,11.813074,37.976353,6.411087,19.448170,0,0,0,0,-3.4,PN,5.2
6,T,K,2.0,0.1550,-0.079163,-0.079163,Benign,0,Benign,0,Benign,0,0.0,0.0,0.0,-0.164003,70.353038,49.220175,21.132863,34.116774,36.236264,0.350520,0.792,3.0,23.845230,28.254788,16.440256,14.313013,0.0,10.719504,17.479466,19.159104,7.792541,6.411087,22.258059,26.465790,29.753970,22.173557,13.683550,16.240129,21.365588,3.755277,18.185293,12.235307,23.640460,22.629094,30.619216,24.601382,39.540737,37.286003,43.923780,25.572965,29.902617,16.440256,20.706048,11.813074,37.976353,6.411087,19.448170,0,0,0,1,2.7,Ppos,-3.2
7,T,P,2.0,0.1940,-0.124339,-0.124339,Benign,0,Benign,0,Benign,0,0.0,0.0,0.0,0.176875,70.353038,49.220175,21.132863,34.116774,36.236264,0.350520,0.792,3.0,23.845230,28.254788,16.440256,14.313013,0.0,10.719504,17.479466,19.159104,7.792541,6.411087,22.258059,26.465790,29.753970,22.173557,13.683550,16.240129,21.365588,3.755277,18.185293,12.235307,23.640460,22.629094,30.619216,24.601382,39.540737,37.286003,43.923780,25.572965,29.902617,16.440256,20.706048,11.813074,37.976353,6.411087,19.448170,0,0,0,0,-0.6,PN,-0.9
8,T,R,2.0,0.0630,-0.222939,-0.222939,Ben

In [488]:
calculatedproperties_df.to_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_CalculatedProperties.csv", index=False)